In [18]:
import pandas as pd
import re
import random
import numpy as np

In [28]:
df = pd.read_csv('bicicletas/AccidentesBicicletas_2017.csv', encoding='latin_1',sep = ';')

In [29]:
def minor_change_2017(df):
    dict_rename = {
        'Nº VICTIMAS *':'* Nº VICTIMAS'
    }
    df.rename(columns=dict_rename, inplace=True)
    return df

df = minor_change_2017(df)

In [30]:
df.head()

,FECHA,RANGO HORARIO,DIA SEMANA,DISTRITO,LUGAR ACCIDENTE,Nº,Nº PARTE,CPFA Granizo,CPFA Hielo,CPFA Lluvia,...,CPSV Grava Suelta,CPSV Hielo,CPSV Seca Y Limpia,* Nº VICTIMAS,TIPO ACCIDENTE,Tipo Vehiculo,TIPO PERSONA,SEXO,LESIVIDAD,Tramo Edad
0,01/01/2017,DE 6:00 A 6:59,DOMINGO,ARGANZUELA,CALLE DE TOLEDO NUM ...,120,2017/88,NO,NO,NO,...,NO,NO,SI,1.0,CHOQUE CON OBJETO FIJO,BICICLETA,CONDUCTOR,MUJER,HG,DE 21 A 24 AÑOS
1,02/01/2017,DE 21:00 A 21:59,LUNES,SAN BLAS,CALLE DE MEQUINENZA NUM ...,14,2017/70,NO,NO,NO,...,NO,NO,SI,1.0,CAÍDA BICICLETA,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 25 A 29 AÑOS
2,03/01/2017,DE 19:00 A 19:59,MARTES,CENTRO,CALLE DE LA ESCALINATA NUM ...,8,2017/307,NO,NO,NO,...,NO,NO,SI,1.0,CAÍDA BICICLETA,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 18 A 20 AÑOS
3,04/01/2017,DE 21:00 A 21:59,MIERCOLES,CENTRO,CALLE DE LA CAVA DE SAN MIGUEL NUM ...,13,2017/124,NO,NO,NO,...,NO,NO,NO,1.0,CAÍDA BICICLETA,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 30 A 34 ANOS
4,05/01/2017,DE 8:00 A 8:59,JUEVES,MONCLOA-ARAVACA,PUENTE DE LOS FRANCESES ...,0,2017/206,NO,NO,NO,...,NO,NO,NO,1.0,CAÍDA BICICLETA,BICICLETA,CONDUCTOR,HOMBRE,HL,DE 45 A 49 AÑOS


In [31]:
def change_cols_17_18(df):
    dict_cols = {
        'FECHA':'dia',
        'DISTRITO':'distrito', 'LUGAR ACCIDENTE': 'calle', 'Nº': 'num',
        'TIPO ACCIDENTE':'tipo_accidente', 'LESIVIDAD': 'lesividad',
        'CPFA Granizo':'Granizo', 'CPFA Hielo': 'Hielo', 'CPFA Lluvia': 'Lluvia',
        'CPFA Niebla': 'Niebla', 'CPFA Seco': 'Despejado', 'CPFA Nieve': 'Nieve',
    }
    to_drop = ['Nº PARTE','CPSV Seca Y Limpia', 'CPSV Mojada', 'CPSV Aceite', 'CPSV Barro', 'CPSV Grava Suelta', 'CPSV Hielo', '* Nº VICTIMAS', 'Tipo Vehiculo', 'TIPO PERSONA', 'SEXO', 'Tramo Edad']
    df.rename(columns=dict_cols, inplace = True)
    df.drop(to_drop, axis=1, inplace=True)
    
    return df

df = change_cols_17_18(df)

In [32]:
    
def create_weather_17_18 (df):
    lst_w = list(df[df.columns[6:12]])
    print(lst_w)
    for e in lst_w:
        df[e] = df[e].apply(lambda x: '' if (x=='NO') else e)
    df['meteo'] = df[lst_w].sum(axis=1)
    df.drop(lst_w, axis=1, inplace=True)
    return df
    
df = create_weather_17_18(df)

['Granizo', 'Hielo', 'Lluvia', 'Niebla', 'Despejado', 'Nieve']


In [33]:
df.head()

,dia,RANGO HORARIO,DIA SEMANA,distrito,calle,num,tipo_accidente,lesividad,meteo
0,01/01/2017,DE 6:00 A 6:59,DOMINGO,ARGANZUELA,CALLE DE TOLEDO NUM ...,120,CHOQUE CON OBJETO FIJO,HG,Despejado
1,02/01/2017,DE 21:00 A 21:59,LUNES,SAN BLAS,CALLE DE MEQUINENZA NUM ...,14,CAÍDA BICICLETA,HL,Despejado
2,03/01/2017,DE 19:00 A 19:59,MARTES,CENTRO,CALLE DE LA ESCALINATA NUM ...,8,CAÍDA BICICLETA,HL,Despejado
3,04/01/2017,DE 21:00 A 21:59,MIERCOLES,CENTRO,CALLE DE LA CAVA DE SAN MIGUEL NUM ...,13,CAÍDA BICICLETA,HL,Niebla
4,05/01/2017,DE 8:00 A 8:59,JUEVES,MONCLOA-ARAVACA,PUENTE DE LOS FRANCESES ...,0,CAÍDA BICICLETA,HL,Niebla


In [34]:
df.meteo.value_counts()

Despejado                                 717
Lluvia                                     32
Niebla                                      5
GranizoHieloLluviaNieblaDespejadoNieve      1
Hielo                                       1
NieblaDespejado                             1
Name: meteo, dtype: int64

In [9]:
def refactor_meteo_17_18 (values):
    dict_meteo = {
        r'Granizo': 'Granizo',
        r'Hielo': 'Nieve',
        r'Nieve': 'Nieve',
        r'Lluvia': 'Lluvia',
        r'Niebla': 'Niebla',}
    for k,v in dict_meteo.items():
        if re.match(k, values):
            return v
    return 'Despejado'

df.meteo = df.meteo.apply(lambda x: refactor_meteo_17_18(x))

In [10]:
df.meteo.value_counts()

Despejado    717
Lluvia        32
Niebla         6
Nieve          1
Granizo        1
Name: meteo, dtype: int64

In [11]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df['fecha'] = df['fecha'].dt.round('H')
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

In [12]:
def create_date_17_18(df):
    df['hora'] = df['RANGO HORARIO'].str.extract(r'(\d[\d]?:\d+\s)')
    create_date(df)
    df.drop(['RANGO HORARIO', 'DIA SEMANA'], axis=1, inplace=True)
    return df

df = create_date_17_18(df)

In [13]:
df.head()

,distrito,calle,num,tipo_accidente,lesividad,meteo,fecha
0,ARGANZUELA,CALLE DE TOLEDO NUM ...,120,CHOQUE CON OBJETO FIJO,HG,Despejado,2017-01-01 06:00:00
1,SAN BLAS,CALLE DE MEQUINENZA NUM ...,14,CAÍDA BICICLETA,HL,Despejado,2017-01-02 21:00:00
2,CENTRO,CALLE DE LA ESCALINATA NUM ...,8,CAÍDA BICICLETA,HL,Despejado,2017-01-03 19:00:00
3,CENTRO,CALLE DE LA CAVA DE SAN MIGUEL NUM ...,13,CAÍDA BICICLETA,HL,Niebla,2017-01-04 21:00:00
4,MONCLOA-ARAVACA,PUENTE DE LOS FRANCESES ...,0,CAÍDA BICICLETA,HL,Niebla,2017-01-05 08:00:00


In [14]:
df.shape

(757, 7)

In [15]:
df.dropna(inplace=True)

In [16]:
df.shape

(756, 7)

In [17]:
def create_dir(df):
    df.num = df.num.apply(lambda x: x.strip())
    df.num = df.num.apply(lambda x: '-' if x=='0' else x)
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion = df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

df = create_dir(df)

In [18]:
def change_accident(values):
    dict_accidente = {
        r'caída':'Caida',
        r'colisión':'Colision',
        r'alcance':'Colision',
        r'atropello':'Imprudencia',
        r'choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values.lower()):
            return v
    return 'Otro'

df.tipo_accidente = df.tipo_accidente.apply(lambda x: change_accident(x))

In [19]:
df.head()

,distrito,tipo_accidente,lesividad,meteo,fecha,direccion
0,ARGANZUELA,Imprudencia,HG,Despejado,2017-01-01 06:00:00,CALLE DE TOLEDO NUM ...
1,SAN BLAS,Caida,HL,Despejado,2017-01-02 21:00:00,CALLE DE MEQUINENZA NUM ...
2,CENTRO,Caida,HL,Despejado,2017-01-03 19:00:00,CALLE DE LA ESCALINATA NUM ...
3,CENTRO,Caida,HL,Niebla,2017-01-04 21:00:00,CALLE DE LA CAVA DE SAN MIGUEL NUM ...
4,MONCLOA-ARAVACA,Caida,HL,Niebla,2017-01-05 08:00:00,PUENTE DE LOS FRANCESES


In [20]:
df.lesividad.value_counts(dropna=False)

HL                                          628
HG                                           60
IL                                           59
NO ASIGNADA                                   8
MT                                            1
Name: lesividad, dtype: int64

In [21]:
def change_injury_17_18(values):
    dict_injury = {
        'IL':'Moderada',
        'HG':'Grave',
        'MT':'Fallecido',
        'HL':'Leve', 'NO ASIGNADA':'Leve'}
    return dict_injury[values]

df.lesividad = df.lesividad.apply(lambda x: change_injury_17_18(x.strip()))

In [22]:
df.head()

,distrito,tipo_accidente,lesividad,meteo,fecha,direccion
0,ARGANZUELA,Imprudencia,Grave,Despejado,2017-01-01 06:00:00,CALLE DE TOLEDO NUM ...
1,SAN BLAS,Caida,Leve,Despejado,2017-01-02 21:00:00,CALLE DE MEQUINENZA NUM ...
2,CENTRO,Caida,Leve,Despejado,2017-01-03 19:00:00,CALLE DE LA ESCALINATA NUM ...
3,CENTRO,Caida,Leve,Niebla,2017-01-04 21:00:00,CALLE DE LA CAVA DE SAN MIGUEL NUM ...
4,MONCLOA-ARAVACA,Caida,Leve,Niebla,2017-01-05 08:00:00,PUENTE DE LOS FRANCESES


In [19]:
def get_df_norm_header_index(path):
    lst_header = ['num_exp', 'dia', 'hora', 'calle', 'num', 'distrito', 'tipo_accidente', 'meteo', 'vehiculo', 'persona', 'edad', 'sexo', 'lesividad' ]
    df = pd.read_csv(path, encoding='latin_1',sep = ';', names=lst_header)
    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

bici = get_df_norm_header_index('bicicletas/AccidentesBicicletas_2019.csv')


In [20]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df['fecha'] = df['fecha'].dt.round('H')
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

bici = create_date(bici)

In [21]:
def create_dir(df):
    df.num.apply(lambda x: x.strip())
    df.num = df.num.apply(lambda x: '-' if x=='0' else x)
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

bici = create_dir(bici)

In [22]:
def fill_drop(df):
    df.lesividad.fillna('14', inplace=True)
    df.meteo.fillna('Se desconoce', inplace=True)
    df.drop(['sexo', 'vehiculo', 'persona', 'edad', 'num_exp'], axis=1, inplace=True)
    return df

bici = fill_drop(bici)

In [23]:
def change_accident(values):
    dict_accidente = {
        r'caída':'Caida',
        r'colisión':'Colision',
        r'alcance':'Colision',
        r'atropello':'Imprudencia',
        r'choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values.lower()):
            return v
    return 'Otro'

bici.tipo_accidente = bici.tipo_accidente.apply(lambda x: change_accident(x))

In [24]:
def change_injury(values):
    dict_injury = {
        '01':'Moderada','02':'Moderada',
        '03':'Grave',
        '04':'Fallecido',
        '05':'Leve','06':'Leve','07':'Leve','14':'Leve'}
    return dict_injury[values]


bici.lesividad = bici.lesividad.apply(lambda x: change_injury(str(x)))

In [25]:
bici = bici[['fecha','meteo', 'tipo_accidente', 'lesividad', 'distrito', 'direccion']]

In [26]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019-01-03 14:00:00,Se desconoce,Colision,Leve,VILLA DE VALLECAS,CALL. FELIPE ALVAREZ 10
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [27]:
bici.meteo.value_counts()

Despejado         703
Se desconoce       30
Lluvia débil       27
Nublado            13
LLuvia intensa      5
Granizando          1
Name: meteo, dtype: int64

In [ ]:
# encoding='cp850'
# encoding='cp1250'

# **
# encoding='latin_1'
# encoding='iso8859_15'

In [18]:
abr = pd.read_csv('tiempo/Anio2019/abr_meteo19.csv', encoding='latin_1',sep = ';')

In [20]:
abr.head(20)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2019,4,1,1.90,V,...,1.45,V,0.90,V,0.97,V,1.68,V,1.12,V
1,28,79,102,81,28079102_81_98,2019,4,2,0.40,V,...,2.48,V,2.05,V,1.88,V,0.95,V,1.10,V
2,28,79,102,81,28079102_81_98,2019,4,3,1.18,V,...,3.95,V,3.53,V,3.40,V,3.42,V,2.58,V
3,28,79,102,81,28079102_81_98,2019,4,4,2.35,V,...,3.12,V,2.52,V,2.30,V,2.13,V,1.88,V
4,28,79,102,81,28079102_81_98,2019,4,5,3.02,V,...,2.88,V,1.43,V,0.93,V,2.18,V,2.58,V
5,28,79,102,81,28079102_81_98,2019,4,6,2.18,V,...,4.93,V,2.98,V,3.07,V,1.95,V,1.83,V
6,28,79,102,81,28079102_81_98,2019,4,7,2.77,V,...,1.73,V,1.48,V,0.73,V,0.37,V,0.75,V
7,28,79,102,81,28079102_81_98,2019,4,8,1.18,V,...,3.95,V,2.82,V,2.50,V,4.33,V,1.30,V
8,28,79,102,81,28079102_81_98,2019,4,9,1.22,V,...,3.48,V,3.92,V,3.15,V,2.98,V,3.67,V
9,28,79,102,81,28079102_81_98,2019,4,10,1.85,V,...,2.02,V,1.65,V,1.68,V,1.73,V,1.60,V
